In [1]:
import pandas as pd
import numpy as np

# librerías de visualización
import seaborn as sns
import matplotlib.pyplot as plt

# librerías para crear el modelo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder  

from sklearn import tree

# para calcular las métricas
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.metrics import precision_score 
from sklearn.metrics import recall_score 
from sklearn.metrics import f1_score 
from sklearn.metrics import cohen_kappa_score


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor 
from sklearn.model_selection import GridSearchCV
from sklearn import tree


from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('data/train_sinprecio.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,-1.045473,3,1,4,0.454922,0.245377,-1.261220,-1.258153,-1.205849,6.353
1,1,0.446332,0,0,7,0.663723,-0.649661,0.613952,0.629644,0.706968,9.183
2,2,-0.162997,0,2,5,0.037321,0.692896,-0.017031,0.006152,0.003726,7.983
3,3,0.593412,2,3,4,1.011723,-0.202142,0.720596,0.664283,0.819486,8.371
4,4,-0.919405,3,3,2,0.385322,0.692896,-1.092366,-1.024343,-1.008941,6.588


In [3]:
X = df.drop(["id",'price'], axis =1)
y = df['price']


X_train, X_test, y_train, y_test = train_test_split(X, y, train_size   = 0.8, random_state = 42)

In [4]:
print("Datos de entrenamiento")
print("-----------------------")
print(y_train.describe())

Datos de entrenamiento
-----------------------
count    32364.000000
mean         7.786349
std          1.018371
min          5.787000
25%          6.851000
50%          7.784000
75%          8.584000
max          9.842000
Name: price, dtype: float64


In [5]:
print("Datos de testeo")
print("-----------------------")
print(y_test.describe())

Datos de testeo
-----------------------
count    8091.000000
mean        7.773714
std         1.010634
min         5.814000
25%         6.851500
50%         7.761000
75%         8.572500
max         9.842000
Name: price, dtype: float64


In [6]:
from sklearn import metrics

In [7]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [metrics.mean_absolute_error(y_test, y_test_pred), metrics.mean_absolute_error(y_train, y_train_pred)],
                'MSE': [metrics.mean_squared_error(y_test, y_test_pred), metrics.mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)), np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))],
                'R2':  [metrics.r2_score(y_test, y_test_pred), metrics.r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

# decision tree

In [9]:
max_features = np.sqrt(len(X_train.columns))
max_features

3.0

In [10]:
param = {
    "max_depth": list(range(1, 36)),
    "min_samples_split": [2, 5, 10, 20],
    "max_features": [1, 2, 3]
}

arbol2 = GridSearchCV(
            estimator=DecisionTreeRegressor(),
            param_grid= param,
            cv=10,
            verbose=0,
            n_jobs = -1,
            return_train_score = True,
            scoring="neg_mean_squared_error")

In [11]:
arbol2.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                       23, 24, 25, 26, 27, 28, 29, 30, ...],
                         'max_features': [1, 2, 3],
                         'min_samples_split': [2, 5, 10, 20]},
             return_train_score=True, scoring='neg_mean_squared_error')

In [12]:
best_tree = arbol2.best_estimator_
best_tree

DecisionTreeRegressor(max_depth=31, max_features=3, min_samples_split=20)

In [13]:
y_pred_test_dt2 = best_tree.predict(X_test)
y_pred_train_dt2 = best_tree.predict(X_train)

In [14]:
dt_results2 = metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, "Decision tree II")

In [16]:
dt_results2

,MAE,MSE,RMSE,R2,set,modelo
0,0.101301,0.020612,0.143568,0.979817,test,Decision tree II
1,0.073551,0.010805,0.103948,0.989581,train,Decision tree II


In [17]:
import pickle
with open('data/tree_sinprecios.pkl', 'wb') as modelo:
        pickle.dump(best_tree, modelo)